# BentoML Example:  Keras Text Classification

BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML. 

This notebook demonstrates how to use BentoML to turn a Keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

This notebook is built based on Keras's IMDB LSTM tutorial [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=keras&ea=keras-text-classification&dt=keras-text-classification)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -q bentoml tensorflow==1.14.0 numpy>=1.16.6

In [3]:
from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
print("Tensorflow Version: %s" % tf.__version__)

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

import bentoml
print("BentoML Version: %s" % bentoml.__version__)

Tensorflow Version: 1.14.0
BentoML Version: 0.8.3+47.g7af5a8f


In [4]:
max_features = 1000
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
batch_size = 300
index_from=3 # word index offset

# Prepare Dataset
Download the IMDB dataset

In [5]:
# A dictionary mapping words to an integer index
imdb.load_data(num_words=max_features)
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+index_from) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown

# Use decode_review to look at original review text in training/testing data
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [6]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, index_from=index_from)

In [7]:
x_train = sequence.pad_sequences(x_train,
                                 value=word_index["<PAD>"],
                                 padding='post',
                                 maxlen=maxlen)

x_test = sequence.pad_sequences(x_test,
                                value=word_index["<PAD>"],
                                padding='post',
                                maxlen=maxlen)

# Model Training & Evaluation

In [8]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         128000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 259,713
Trainable params: 259,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3, # for demo purpose :P
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 10s 419us/sample - loss: 0.4412 - acc: 0.8008 - val_loss: 0.4132 - val_acc: 0.8128
Epoch 2/3
25000/25000 [==============================] - 10s 385us/sample - loss: 0.4187 - acc: 0.8144 - val_loss: 0.4133 - val_acc: 0.8132
Epoch 3/3
25000/25000 [==============================] - 10s 380us/sample - loss: 0.4122 - acc: 0.8169 - val_loss: 0.4100 - val_acc: 0.8136


In [12]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 2s 80us/sample - loss: 0.4100 - acc: 0.8136
Test score: 0.40997193634510043
Test accuracy: 0.81356


## Define BentoService for model serving

In [32]:
%%writefile keras_text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence, text
from bentoml import api, env, BentoService, artifacts
from bentoml.frameworks.keras import KerasModelArtifact
from bentoml.service.common_artifacts import PickleArtifact
from bentoml.adapters import JsonInput


max_features = 1000

@artifacts([
    KerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(pip_dependencies=['tensorflow==1.14.0', 'numpy', 'pandas'])
class KerasTextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index and self.artifacts.word_index[word] <= max_features:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    def preprocessing(self, text_str):
        sequence = text.text_to_word_sequence(text_str)
        return list(map(self.word_to_index, sequence))
    
    @api(input=JsonInput())
    def predict(self, parsed_jsons):
        input_datas = [self.preprocessing(parsed_json['text']) for parsed_json in parsed_jsons]
        input_datas = sequence.pad_sequences(input_datas,
                                             value=self.artifacts.word_index["<PAD>"],
                                             padding='post',
                                             maxlen=80)

        return self.artifacts.model.predict_classes(input_datas).T[0]

Overwriting keras_text_classification_service.py


## Save BentoService to file archive

In [33]:
# 1) import the custom BentoService defined above
from keras_text_classification_service import KerasTextClassificationService

# 2) `pack` it with required artifacts
bento_svc = KerasTextClassificationService()
bento_svc.pack('model', model)
bento_svc.pack('word_index', word_index)

# 3) save your BentoSerivce
saved_path = bento_svc.save()

[2020-08-04 12:52:19,102] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.8.3+47.g7af5a8f
creating BentoML-0.8.3+47.g7af5a8f/BentoML.egg-info
creating BentoML-0.8.3+47.g7af5a8f/bentoml
creating BentoML-0.8.3+47.g7af5a8f/bentoml/adapters
creating BentoML-0.8.3+47.g7af5a8f/bentoml/artifact
creating BentoML-0.8.3+47.g7af5a8f/bentoml/cli
creating BentoML-0.8.3+47.g7af5a8f/bentoml/clipper
creating BentoML-0.8.3+47.g7af5a8f/bentoml/configuration
creating BentoML-0.8.3+47.g7af5a8f/bentoml/configuration/__pycache__
creating BentoML-0.8.3+47.g7af5a8f/bentoml/handlers
creating BentoML-0.8.3+47.g7af5a8f/bentoml/marshal
creating BentoML-0.8.3+47.g7af5a8f/bentoml/saved_bundle
creating BentoML-0.8.3+47.g7af5a8f/bentoml/server
creating BentoML-0.8.3+47.g7af5a8f/bentoml/utils
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/client
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/deployment
creating BentoML-0.8.3+47.g7af5a8f/bentoml/yatai/dep

## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [3]:
!bentoml serve KerasTextClassificationService:latest

[2020-08-04 12:55:23,028] INFO - Getting latest version KerasTextClassificationService:20200804125206_4639A5
[2020-08-04 12:55:23,028] INFO - Starting BentoML API server in development mode..
[2020-08-04 12:55:24,431] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 12:55:24,450] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g7af5a8f
2020-08-04 12:55:25.644085: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-08-04 12:55:25.663186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA no

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve KerasTextClassificationService:latest --run-with-ngrok

### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '{"text": "best movie ever"}' \
localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [ ]:
!bentoml containerize KerasTextClassificationService:latest

In [48]:
!docker run -p 5000:5000 kerastextclassificationservice

[2020-02-12 19:57:53,686] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-02-12 19:57:54 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-02-12 19:57:54 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-02-12 19:57:54 +0000] [1] [INFO] Using worker: sync
[2020-02-12 19:57:54 +0000] [8] [INFO] Booting worker with pid: 8
[2020-02-12 19:57:54 +0000] [9] [INFO] Booting worker with pid: 9
[2020-02-12 19:57:54 +0000] [10] [INFO] Booting worker with pid: 10
[2020-02-12 19:57:54,287] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,411] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,451] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [15]:
import bentoml

loaded_bento_svc = bentoml.load(saved_path)

loaded_bento_svc.predict([{ "text": "the best movie I have ever seen" }])

array([[1]], dtype=int32)

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [2]:
!bentoml run KerasTextClassificationService:latest predict --input='{"text": "bad movie"}'

[2020-08-04 12:54:48,270] INFO - Getting latest version KerasTextClassificationService:20200804125206_4639A5
[2020-08-04 12:54:49,229] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 12:54:49,260] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g7af5a8f
2020-08-04 12:54:50.476102: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-08-04 12:54:50.491479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-04 12:54:50.491867: I tensorflow/core/common_runtime/gpu/gpu_device

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)